In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import numpy as np
import nltk
import math
from collections import Counter 
from math import log2
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%run data_intake.ipynb

In [6]:
water_resource_df = pipeline("total_exploitable_water_resource_by_year_by_country.xlsx", 
                             "Attainable_yields.csv", "value")
water_resource_df.rename(columns = {'value':'total_exploitable_water_resources'}, inplace = True)

water_resource_df

,Country,Year,total_exploitable_water_resources,barley_crop_yield,cassava_crop_yield,cotton_crop_yield,groundnut_crop_yield,maize_crop_yield,millet_crop_yield,oilpalm_crop_yield,potato_crop_yield,rapeseed_crop_yield,rice_crop_yield,rye_crop_yield,sorghum_crop_yield,soybean_crop_yield,sugarbeet_crop_yield,sugarcane_crop_yield,sunflower_crop_yield,wheat_crop_yield
0,Albania,2018,13.0,2.84,NaN,1.11,NaN,7.23,NaN,NaN,26.17,NaN,NaN,2.31,NaN,2.55,39.15,NaN,2.09,3.69
1,Albania,2017,13.0,2.95,NaN,1.11,NaN,6.56,NaN,NaN,25.11,NaN,NaN,2.27,NaN,2.20,23.48,NaN,2.09,4.04
2,Albania,2016,13.0,2.90,NaN,1.11,NaN,6.49,NaN,NaN,24.59,NaN,NaN,2.25,NaN,2.52,22.61,NaN,2.09,3.90
3,Albania,2015,13.0,2.80,NaN,1.11,NaN,6.96,NaN,NaN,24.26,NaN,NaN,2.31,NaN,2.50,25.06,NaN,2.09,3.95
4,Albania,2014,13.0,2.70,NaN,1.11,NaN,6.91,NaN,NaN,25.00,NaN,NaN,2.23,NaN,1.69,22.77,NaN,2.09,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,Zimbabwe,1968,1.5,2.79,3.06,1.16,0.46,1.32,0.53,NaN,12.50,NaN,1.52,NaN,0.41,1.73,NaN,104.85,0.61,3.58
2261,Zimbabwe,1967,1.5,2.79,3.03,1.94,0.70,1.77,0.47,NaN,13.06,NaN,1.40,NaN,0.79,1.67,NaN,105.33,0.65,2.69
2262,Zimbabwe,1966,1.5,2.79,3.03,1.47,0.65,1.14,0.54,NaN,10.95,NaN,1.30,NaN,0.80,1.77,NaN,84.82,0.55,2.01
2263,Zimbabwe,1965,1.5,2.79,3.01,1.18,0.35,1.10,0.65,NaN,13.38,NaN,1.30,NaN,0.69,0.46,NaN,109.85,0.51,2.35


In [9]:
def total(cnt):
    return sum(cnt.values())

def gini(cnt):
    tot = total(cnt)
    return 1 - sum([(v/tot)**2 for v in cnt.values()])

def entropy(cnt):
    tot = total(cnt)
    return sum([(-v/tot) * log2(v/tot) for v in cnt.values()])
    
def wavg(cnt1, cnt2, measure):
    tot1 = total(cnt1)
    tot2 = total(cnt2)
    tot = tot1 + tot2
    return (measure(cnt1) * tot1 + measure(cnt2) * tot2) / tot

def evaluate_split(df, class_col, split_col, feature_val, measure):
    df1, df2 = df[df[split_col] == feature_val], df[df[split_col] != feature_val]
    cnt1, cnt2 = Counter(df1[class_col]), Counter(df2[class_col])
    return wavg(cnt1, cnt2, measure)


def split(df, split_col, v):
    """
    Subdivides the dataframe based on the choosen split_column and value partition
    """
    left_split = df[df[split_col] <= v]
    right_split = df[df[split_col] > v]
    return left_split, right_split
    


def get_best_split(df, class_col, method, target_impurity):
    """
    Returns information for a split of the dataset based on the choosen method (gini or other),
    including the right and left partitions of the dataframe
    """
    # Holder variables for the information regarding our best split of the df
    best_col = 0
    best_v = ''
    best_meas = float("inf")
    best_split_left = None
    best_split_right = None
    
    # Loop through all the feature columns
    for split_col in df.columns:
        # Make sure it's not the class column
        if split_col != class_col:
            # Find all the unique values in the feature column
            for v in set(df[split_col]):
                # Create a split w/ the values greater than or equal to or less than for left and right splits
                split_left, split_right = split(df, split_col, v)
                # important step, stops infinite recursion on the same split
                if len(split_left)>0 and len(split_right)>0:
                    # Evaluate this split w/ the given metric
                    meas = evaluate_split(df, class_col, split_col, v, method)
                    # If we found a better split (and the impurity is okay), we set it as our best split
                    # and update the holder variables accordingly
                    if meas < best_meas and meas > target_impurity:
                        best_meas = meas
                        best_col = split_col
                        best_v = v
                        best_split_left = split_left
                        best_split_right = split_right

    return best_col, best_v, best_meas, best_split_left, best_split_right
                    


def compute_majority_class(df, class_col):
    """
    Compute the majority class for this node
    """
    # Count the number of each unique class in the class column
    counts = Counter(df[class_col])
    majority_class = None
    # Holder variable to keep track of the number of occurences of the majority class
    max_count = 0
    # Iterate through the counts tuple, find the majority class
    for k,v in counts.items():
        if v > max_count:
            max_count = v
            majority_class = k
    return majority_class
        

In [8]:
def dtree(train, criterion, max_depth=None, current_depth=0, min_instances=2, target_impurity=0.0, class_col_name):
    """
    Constructs binary decision tree based on the input parameters
    """
    # If the input dataset is empty at this point in recursion, we have hit a return leaf
    if train is None or len(train) == 0:
        return None
    # If the input dataset is smaller than our min number of instances or the max depth has been exceeded, return leaf
    elif len(train) < min_instances or current_depth == max_depth:
        return None
    
    # Generate the best split for this input dataset
    best_col, best_v, best_meas, left_subtree, right_subtree = get_best_split(
        train, 'track_genre', criterion, target_impurity)
    
    # The split with the best measusure must have had an empty side, so we have to terminate to a leaf  
    # Or no best measure was found (stops infinite recursion)
    if best_meas == float("inf"):
        return None

    # Recursive formulation of building the decision tree
    return (best_col, best_v, len(train), compute_majority_class(train, 'track_genre'), best_meas, current_depth,
            dtree(left_subtree, criterion, max_depth, current_depth + 1, min_instances, target_impurity), 
            dtree(right_subtree, criterion, max_depth, current_depth + 1, min_instances, target_impurity))


In [7]:
test_tree = dtree(water_resource_df[0:200], gini, max_depth=None, target_impurity=0.0, "class_col_name")
test_tree

NameError: name 'dtree' is not defined